# Step 1. We need the football data. 

We have several ways to obtain the football data:

1. Paying to the suppliers of data. For example, in Spain we can pay Opta for the data.
2. Sometimes we get data from suppliers or some specific pages. 
For example, we search on Google **statsbomb open data**

[statsbomb open data](https://github.com/statsbomb/open-data)

The other site to search for data is **wyscout**. Wyscout has their data in a data repository called Figshare, so I'm going to search:

**figshare wyscout**

[figshare wyscout](https://figshare.com/collections/Soccer_match_event_dataset/4415000/2)

Other places where we can find data are:

- [Kaggle](https://www.kaggle.com) 

- [Northern Arizona U](https://libraryguides.nau.edu/c.php?g=410999&p=2801309) 

- [data.world](https://data.world/datasets/sports)


3. **Web scraping** 
But we can also do some searching and web scraping of a page like:

[fbref](https://fbref.com/en/comps/12/La-Liga-Stats)



### Web scraping plan:

1. Make the request using the library `request` and using `get`
2. Parser the HTML with the library `BeautifulSoup`
3. Find the table using `soup`
4. Create a DataFrame with the extracted data




In [39]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


# FBRef website URL for La Liga
url = "https://fbref.com/en/comps/12/La-Liga-Stats"

# Make the GET request to the web page
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the web page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Show a part of the HTML content to verify
    print(soup.prettify()[:1000])  # Show the first 1000 characters of the HTML
else:
    print(f"Error accessing the page: {response.status_code}")

<!DOCTYPE html>
<html class="no-js" data-root="/home/fb/deploy/www/base" data-version="klecko-" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport">
   <link href="https://cdn.ssref.net/req/202406241" rel="dns-prefetch"/>
   <script>
    /* https://docs.osano.com/hc/en-us/articles/22469433444372-Google-Consent-Mode-v2  */
  window.dataLayer = window.dataLayer ||[];
      function gtag(){dataLayer.push(arguments);}
      gtag('consent','default',{
        'ad_storage':'denied',
        'analytics_storage':'denied',
        'ad_user_data':'denied',
        'ad_personalization':'denied',
        'personalization_storage':'denied',
        'functionality_storage':'granted',
        'security_storage':'granted',
        'wait_for_update': 500
      });
      gtag("set", "ads_data_redaction", true);
   </script>
   <script src="https://cmp.osano.com/16C

Normally these tables are in an HTML term called `table`, so I have asked **ChatGPT** to make me a program to search all the tables in the HTML

In [40]:
# FBRef website URL for La Liga
url = "https://fbref.com/en/comps/12/La-Liga-Stats"

# Make the GET request to the web page
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the web page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all tables in the HTML
    tables = soup.find_all('table')

    # Show information about each table found
    for i, table in enumerate(tables):
        print(f"Tabla {i}:")
        print(table.prettify()[:500])  # Show the first 500 characters of each table
        print("\n\n")

    # Here you could manually inspect the outputs to identify the correct table.

else:
    print(f"Error al acceder a la página: {response.status_code}")

Tabla 0:
<table class="stats_table sortable min_width force_mobilize" data-cols-to-freeze=",2" id="results2023-2024121_overall">
 <caption>
  La Liga Table
 </caption>
 <colgroup>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
 </colgroup>
 <thead>
  <tr>
   <th aria-label="Rank" class="poptip sort_default_asc center" data-stat="rank" data-tip="&lt;strong&gt;Rank&lt;/strong&gt;&lt;b



Tabla 1:
<table class="stats_table sortable min_width force_mobilize" data-cols-to-freeze=",2" id="results2023-2024121_home_away">
 <caption>
  La Liga Table
 </caption>
 <colgroup>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
 </colgroup>
 <thead>
  <tr class="over_header">
   <

In [48]:

# URL of the FBRef page for La Liga
url = "https://fbref.com/en/comps/12/La-Liga-Stats"

# Make the GET request to the web page
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the web page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the statistics table by id
    table = soup.find('table', {'id': 'results2023-2024121_overall'})

    # Check if the table was found
    if table:
        # Extract the headers of the table
        headers = []
        for th in table.find_all('th')[1:]:  # Skip the first header
            headers.append(th.get_text().strip())

        # Extract the rows of the table
        rows = []
        for row in table.find_all('tr')[1:]:
            cells = row.find_all('td')
            if len(cells) > 0:
                rows.append([cell.get_text().strip() for cell in cells])

        # Find the maximum number of columns in rows
        max_columns = max(len(row) for row in rows)

        # Adjust headers to match the number of columns in rows
        if len(headers) > max_columns:
            headers = headers[:max_columns]
        elif len(headers) < max_columns:
            # If there are more columns in rows, add missing headers
            headers.extend([''] * (max_columns - len(headers)))

        # Insert a header for the ranking column
        headers.insert(0, 'RK')

        # Add the ranking data to each row
        for i, row in enumerate(rows):
            row.insert(0, str(i + 1))

        # Create a DataFrame with the extracted data
        df = pd.DataFrame(rows, columns=headers)

        # Display the DataFrame
        print(df)
    else:
        print("The table with the specified id was not found.")
else:
    print(f"Error accessing the page: {response.status_code}")

    RK            Squad  MP   W   D   L  GF  GA   GD Pts Pts/MP    xG   xGA  \
0    1      Real Madrid  38  29   8   1  87  26  +61  95   2.50  68.8  35.4   
1    2        Barcelona  38  26   7   5  79  44  +35  85   2.24  77.6  41.6   
2    3           Girona  38  25   6   7  85  46  +39  81   2.13  70.8  54.3   
3    4  Atlético Madrid  38  24   4  10  70  43  +27  76   2.00  62.4  39.5   
4    5    Athletic Club  38  19  11   8  61  37  +24  68   1.79  52.7  41.3   
5    6    Real Sociedad  38  16  12  10  51  39  +12  60   1.58  44.9  41.3   
6    7            Betis  38  14  15   9  48  45   +3  57   1.50  43.9  50.2   
7    8       Villarreal  38  14  11  13  65  65    0  53   1.39  55.6  58.7   
8    9         Valencia  38  13  10  15  40  45   -5  49   1.29  43.1  47.6   
9   10           Alavés  38  12  10  16  36  46  -10  46   1.21  46.2  44.7   
10  11          Osasuna  38  12   9  17  45  56  -11  45   1.18  40.5  49.9   
11  12           Getafe  38  10  13  15  42  54  -12

## Glossary of our dataset
1. **Rk** -- Rank
2. **MP** -- Matches Played: Matches Played by the player or squad
3. **W** -- Wins: Wins
4. **D** -- Draws: Draws
5. **L** -- Losses: Losses
6. **GF** -- Goals For: Goals For
7. **GA** -- Goals Against: Goals Against
8. **GD** -- Goal Difference: Goal Difference
9. **Pts** -- Points:Most leagues are ordered by points. Three for a win and one for a draw.
10. **Pts/MP** -- Points/Match: Points per Match Played
11. **xG** -- Expected Goals: xG totals include penalty kicks, but do not include penalty shootouts (unless otherwise noted). Provided by Opta. An underline indicates there is a match that is missing data, but will be updated when available.
12. **xGA** -- xG Allowed: Expected Goals Allowed. xG totals include penalty kicks, but do not include penalty shootouts (unless otherwise noted). Provided by Opta. An underline indicates there is a match that is missing data, but will be updated when available.
13. **xGD** -- xG Difference: Expected Goals Difference. xG totals include penalty kicks, but do not include penalty shootouts (unless otherwise noted). Provided by Opta. An underline indicates there is a match that is missing data, but will be updated when available.
14. **xGD/90** -- xG Difference/90: Expected Goals Difference per 90 Minutes. xG totals include penalty kicks, but do not include penalty shootouts (unless otherwise noted). Provided by Opta. An underline indicates there is a match that is missing data, but will be updated when available.
15. **Attendance** -- Attendance/Game: Attendance per game during this season, only for home matches.
16. **Top Team Scorer** -- Only includes league matches for that season
17. **Goalkeeper** -- Goalkeeper with the most minutes in league play

In [49]:
df


,RK,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,Notes
0,1,Real Madrid,38,29,8,1,87,26,+61,95,2.50,68.8,35.4,+33.4,+0.88,"72,061",Jude Bellingham - 19,Andriy Lunin,→ Champions League via league finish
1,2,Barcelona,38,26,7,5,79,44,+35,85,2.24,77.6,41.6,+35.9,+0.94,"39,846",Robert Lewandowski - 19,Marc-André ter Stegen,→ Champions League via league finish
2,3,Girona,38,25,6,7,85,46,+39,81,2.13,70.8,54.3,+16.5,+0.43,"12,520",Artem Dovbyk - 24,Paulo Gazzaniga,→ Champions League via league finish
3,4,Atlético Madrid,38,24,4,10,70,43,+27,76,2.00,62.4,39.5,+23.0,+0.60,"59,121",Antoine Griezmann - 16,Jan Oblak,→ Champions League via league finish
4,5,Athletic Club,38,19,11,8,61,37,+24,68,1.79,52.7,41.3,+11.4,+0.30,"46,112",Gorka Guruzeta - 14,Unai Simón,→ Europa League via cup win
5,6,Real Sociedad,38,16,12,10,51,39,+12,60,1.58,44.9,41.3,+3.6,+0.10,"31,710",Mikel Oyarzabal - 9,Álex Remiro,→ Europa League via league finish
6,7,Betis,38,14,15,9,48,45,+3,57,1.50,43.9,50.2,-6.3,-0.17,"51,259",Willian José - 10,Rui Silva,→ Europa Conference League via league finish
7,8,Villarreal,38,14,11,13,65,65,0,53,1.39,55.6,58.7,-3.2,-0.08,"17,957",Alexander Sørloth - 23,Filip Jørgensen,
8,9,Valencia,38,13,10,15,40,45,-5,49,1.29,43.1,47.6,-4.5,-0.12,"43,420",Hugo Duro - 13,Giorgi Mamardashvili,
9,10,Alavés,38,12,10,16,36,46,-10,46,1.21,46.2,44.7,+1.5,+0.04,"17,391",Samuel Omorodion - 8,Antonio Sivera,


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   RK               20 non-null     object
 1   Squad            20 non-null     object
 2   MP               20 non-null     object
 3   W                20 non-null     object
 4   D                20 non-null     object
 5   L                20 non-null     object
 6   GF               20 non-null     object
 7   GA               20 non-null     object
 8   GD               20 non-null     object
 9   Pts              20 non-null     object
 10  Pts/MP           20 non-null     object
 11  xG               20 non-null     object
 12  xGA              20 non-null     object
 13  xGD              20 non-null     object
 14  xGD/90           20 non-null     object
 15  Attendance       20 non-null     object
 16  Top Team Scorer  20 non-null     object
 17  Goalkeeper       20 non-null     obje

In [53]:
# column list to convert
columns_to_convert = ['MP','W',	'D','L','GF','GA','GD','Pts','Pts/MP','xG','xGA','xGD','xGD/90','Attendance']

# Convert all columnas in the list to numeric
for col in columns_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Verify the conversion
print(df.dtypes)

RK                  object
Squad               object
MP                   int64
W                    int64
D                    int64
L                    int64
GF                   int64
GA                   int64
GD                   int64
Pts                  int64
Pts/MP             float64
xG                 float64
xGA                float64
xGD                float64
xGD/90             float64
Attendance         float64
Top Team Scorer     object
Goalkeeper          object
Notes               object
dtype: object
